In [ ]:
import os

path,dirs,files = next(os.walk('train'))

print('There is')
print(len(files))
print('files')

In [ ]:
file_names = os.listdir('train')[:2000]
print(file_names[0])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
cat = mpimg.imread('train/cat.10145.jpg')
plt.imshow(cat)
plt.show()

In [ ]:
dog = mpimg.imread('train/dog.10145.jpg')
plt.imshow(dog)
plt.show()

In [ ]:
dog_files = []
cat_files = []

labels = []

for file_name in file_names:
    if file_name.startswith('cat'):
        cat_files.append(file_name)

        labels.append(0)
    elif file_name.startswith('dog'):
        dog_files.append(file_name)

        labels.append(1)
        
print(len(dog_files))
print(len(cat_files))

In [ ]:
import PIL

for file_name in file_names:
    img = PIL.Image.open('train/'+ file_name)
    img = img.resize((224,224))
    img = img.convert('RGB')

    img.save('resized/'+ file_name)

In [ ]:
import cv2
import glob
import numpy as np

catdogimgs = []

for file_name in file_names:
    catdogimgs.append(cv2.imread('resized/'+ file_name))

catdogimgs = np.asarray(catdogimgs)


In [ ]:
from sklearn.model_selection import train_test_split

labels = np.asarray(labels)

X_train, X_test, Y_train, Y_test = train_test_split(catdogimgs, labels, test_size = 0.2, random_state = 2)


X_train_scaled = X_train/255

X_test_scaled = X_test/255

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

mobilenet_model = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

pretrained_model = hub.KerasLayer(mobilenet_model, input_shape=(224,224,3), trainable=False)

In [ ]:
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Dense(2)
])

model.summary()

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

In [ ]:
model.fit(X_train_scaled, Y_train, epochs = 5)

In [ ]:
score, acc = model.evaluate(X_test_scaled, Y_test)
print('Test Loss =', score)
print('Test Accuracy =', acc)

In [ ]:
image_number = input("Give me a number between 1 and 12500")

test_image = mpimg.imread(f'test1/{image_number}.jpg')
plt.imshow(test_image)
plt.show()

In [ ]:
input_image = cv2.imread(f'test1/{image_number}.jpg')

input_image_resize = cv2.resize(input_image, (224,224))

input_image_scaled = input_image_resize/255

image_reshaped = np.reshape(input_image_scaled, [1,224,224,3])

input_prediction = model.predict(image_reshaped)

print(input_prediction)

In [ ]:
catdogimgs = np.argmax(input_prediction)

print(catdogimgs)

if catdogimgs == 0:
    print('😺')
elif catdogimgs == 1:
    print('🐶')